# Exercise: Teach an LLM to Spell with Group Relative Policy Optimization (GRPO)

Large language models (LLMs) are notoriously bad at spelling. This is partly because tokenizers break words into smaller pieces, so the model learns about sub-word units rather than whole words and their spellings.

In this exercise, you'll use Group Relative Policy Optimization (GRPO) and a technique called Parameter-Efficient Fine-Tuning (PEFT) with Low-Rank Adaptation (LoRA) to teach a small LLM how to spell words. This is a classic example of teaching a model a new skill that isn't well-represented in its pre-training data.

## What you'll do in this notebook

1.  **Setup**: Import libraries and configure the environment.
2.  **Load the tokenizer and base model**: Use a small, instruction-tuned model as our starting point.
3.  **Create the dataset**: Generate a simple dataset of words and their correct spellings.
4.  **Evaluate the base model**: Test the model's spelling ability *before* fine-tuning to establish a baseline.
5.  **Configure LoRA and train**: Attach a LoRA adapter to the model and fine-tune it on the spelling dataset.
6.  **Evaluate the fine-tuned model**: Test the model again to see if its spelling has improved.

### The core design pattern (memorize this)

SFT datasets contain answers --> 
GRPO datasets contain graders

## Setup

In [1]:
# Setup imports
# No changes needed in this cell

import os
import torch
from datasets import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig

# Use GPU, MPS, or CPU, in that order of preference
if torch.cuda.is_available():
    device = torch.device("cuda")  # NVIDIA GPU
elif torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple Silicon
else:
    device = torch.device("cpu")
torch.set_num_threads(max(1, os.cpu_count() // 2))
print("Using device:", device)

/Users/xina/dev/Spam_Classifier/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: mps


## Step 1. Load the tokenizer and base model

The model `HuggingFaceTB/SmolLM2-135M-Instruct` is a small, instruction-tuned model that's suitable for this exercise. It has 135 million parameters, making it lightweight and efficient for fine-tuning. It's not the most powerful model, but it's a good choice for demonstrating the concepts of SFT and PEFT with LoRA, especially on a CPU or limited GPU resources.

In [19]:
# Student task: Load the model and tokenizer, and copy the model to the device.
# TODO: Complete the sections with **********

# See: https://huggingface.co/docs/transformers/en/models
# See: https://huggingface.co/docs/transformers/en/fast_tokenizers

# Model ID for SmolLM2-135M-Instruct
model_id = "HuggingFaceTB/SmolLM2-135M-Instruct"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_id)

# Copy the model to the device (GPU, MPS, or CPU)
model = model.to(device)


print("Model parameters (total):", sum(p.numel() for p in model.parameters()))

Model parameters (total): 134515008


## Step 2. Create the dataset

In [20]:
# Create a list of words of different lengths
# No changes are needed in this cell.

# fmt: off
ALL_WORDS = [
    "idea", "glow", "rust", "maze", "echo", "wisp", "veto", "lush", "gaze", "knit", "fume", "plow",
    "void", "oath", "grim", "crisp", "lunar", "fable", "quest", "verge", "brawn", "elude", "aisle",
    "ember", "crave", "ivory", "mirth", "knack", "wryly", "onset", "mosaic", "velvet", "sphinx",
    "radius", "summit", "banner", "cipher", "glisten", "mantle", "scarab", "expose", "fathom",
    "tavern", "fusion", "relish", "lantern", "enchant", "torrent", "capture", "orchard", "eclipse",
    "frescos", "triumph", "absolve", "gossipy", "prelude", "whistle", "resolve", "zealous",
    "mirage", "aperture", "sapphire",
]
# fmt: on

In [31]:
# Student Task: Create a Hugging Face Dataset with the prompt that asks the model to spell the word
# with hyphens between the letters.
# TODO: Complete the sections with **********



def generate_records():
    for word in ALL_WORDS:
        yield {
            # We will use the GRPOTrainer which expects to receieve formatted prompts
            # to pass to the LLM
            # https://huggingface.co/docs/trl/main/en/grpo_trainer
            # "**********": f"**********",
            # Before using GRPOTrainer, will run a few epochs of supervised-fine tuning (SFT)
            # which can be useful to give an initial nudge to the model. Thus we need to provide
            # the gold standard answer.
            # See the documentation for more details:
            # https://huggingface.co/docs/trl/en/sft_trainer#expected-dataset-type-and-format
            # "**********": "-".join(word).upper() + ".",
            # GRPOTrainer does not expect a completion, but we can add extra columns to our dataset
            # that our reward functions will use to grade the completions provided by the LLM.
            # "prompt": f"Spell the word '{word}' with hyphens between the letters.",
             "prompt": (
                f"You spell words with hyphens between the letters like this W-O-R-D.\nWord:\n{word}\n\n"
                + "Spelling:\n"
            ),
            "completion":"-".join(word).upper()+".",
            "word": word,
            "Spelling": "-".join(word).upper(),
        }


ds = Dataset.from_generator(generate_records)

ds = ds.train_test_split(test_size=0.1, seed=42)

# Show the first item of the train split
ds["train"][0]

Generating train split: 62 examples [00:00, 7145.13 examples/s]


{'prompt': 'You spell words with hyphens between the letters like this W-O-R-D.\nWord:\ntriumph\n\nSpelling:\n',
 'completion': 'T-R-I-U-M-P-H.',
 'word': 'triumph',
 'Spelling': 'T-R-I-U-M-P-H'}

In [32]:
ds["train"]

Dataset({
    features: ['prompt', 'completion', 'word', 'Spelling'],
    num_rows: 55
})

## Step 3. Evaluate the base model

Before we fine-tune the model, let's see how it performs on the spelling task. We'll create a helper function to generate a spelling for a given word and compare it to the correct answer.

In [33]:
# Create a helper function that will help us visualize the performance of the model
# No changes needed in this cell


def check_spelling(
    model, tokenizer, prompt: str, actual_spelling: str, max_new_tokens: int = 20
) -> (str, str):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    gen = model.generate(
        **inputs, max_new_tokens=max_new_tokens, use_cache=False
    )  # No parameters = greedy search
    output = tokenizer.decode(gen[0], skip_special_tokens=True)

    # Extract the generated spelling from the full output string
    proposed_spelling = output.split("Spelling:\n")[-1].strip().split("\n")[0].strip()

    # strip any whitepsace from the actual spelling
    actual_spelling = actual_spelling.strip()

    print(
        f"Proposed: {proposed_spelling} | Actual: {actual_spelling} "
        f"| Matches: {'✅' if proposed_spelling == actual_spelling else '❌'}"
    )

    # Remove hyphens for a character-by-character comparison
    proposed_spelling = proposed_spelling.replace("-", "")
    actual_spelling = actual_spelling.replace("-", "")

    # Calculate the proportion of the spelling that was correct
    num_correct = sum(1 for a, b in zip(actual_spelling, proposed_spelling) if a == b)

    return num_correct / len(actual_spelling)  # Return proportion correct


In [34]:
# Student task: Evaluate the base model's spelling ability
# We expect it to perform poorly, as it hasn't been trained for this task.

proportion_correct = 0.0

for example in ds["train"].select(range(20)):
    prompt = example["prompt"]
    spelling = example["Spelling"]
    result = check_spelling(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        actual_spelling=spelling,
        max_new_tokens=20,
    )
    proportion_correct += result

print(f"{proportion_correct}/20.0 words correct")

/Users/xina/dev/Spam_Classifier/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Proposed: T-O-R-M-P-I-Z. | Actual: T-R-I-U-M-P-H | Matches: ❌
Proposed: S-A-P-O-R-I-C-H | Actual: S-A-P-P-H-I-R-E | Matches: ❌
Proposed: -E-X-P-S-H-R. | Actual: E-X-P-O-S-E | Matches: ❌
Proposed: F-R-S-O-C-S. | Actual: F-R-E-S-C-O-S | Matches: ❌
Proposed: W-O-R-D. | Actual: W-I-S-P | Matches: ❌
Proposed: - M-I-R-A-G. | Actual: M-I-R-A-G-E | Matches: ❌
Proposed: I-V-O-R-Y. | Actual: I-V-O-R-Y | Matches: ❌
Proposed: - O-N-S-H-R-D. | Actual: O-N-S-E-T | Matches: ❌
Proposed: -E-L-U-D. | Actual: E-L-U-D-E | Matches: ❌
Proposed: -S-P-H-I-N-E | Actual: S-P-H-I-N-X | Matches: ❌
Proposed: B-R-W-N-R. | Actual: B-R-A-W-N | Matches: ❌
Proposed: G-O-S-H-O-P-I. | Actual: G-O-S-S-I-P-Y | Matches: ❌
Proposed: -E-N-C-O-N-T-H. | Actual: E-N-C-H-A-N-T | Matches: ❌
Proposed: T-A-W-R-A-N. | Actual: T-A-V-E-R-N | Matches: ❌
Proposed: - W-O-R-D | Actual: W-H-I-S-T-L-E | Matches: ❌
Proposed: - C-U-R-U-D. | Actual: C-A-P-T-U-R-E | Matches: ❌
Proposed: -E-C-O-R-H. | Actual: E-C-H-O | Matches: ❌
Proposed: M-I-R-

As expected, the base model is terrible at spelling. It mostly just repeats the word back. Now, let's fine-tune it.

## Step 4. Configure LoRA and train the model

Let’s attach a LoRA adapter to the base model. We use a LoRA config so only a tiny fraction of parameters are trainable. Read more here: [LoRA](https://huggingface.co/docs/peft/main/en/conceptual_guides/lora).

In [35]:
# Student task: Configure LoRA for a causal LM and wrap the model with get_peft_model
# Complete the sections with **********

# Print how many params are trainable at first
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(
    f"Trainable params BEFORE: {trainable:,} / {total:,} ({100 * trainable / total:.2f}%)"
)

# See: https://huggingface.co/docs/peft/package_reference/lora
# lora_config = LoraConfig(
#     r=**********,                 # Rank of the update matrices. Lower value = fewer trainable parameters.
#     lora_alpha=**********,        # LoRA scaling factor.
#     lora_dropout=**********,      # Dropout probability for LoRA layers.
#     bias="none",
#     task_type=**********,         # Causal Language Modeling.
# )
# # Wrap the base model with get_peft_model
# model = get_peft_model(**********, **********)

lora_config = LoraConfig(r=64, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")
# Wrap the base model with get_peft_model
model = get_peft_model(model,lora_config)

# Print the number of trainable parameters after applying LoRA
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(
    f"Trainable params AFTER: {trainable:,} / {total:,} ({100 * trainable / total:.2f}%)"
)

Trainable params BEFORE: 3,686,400 / 138,201,408 (2.67%)
Trainable params AFTER: 3,686,400 / 138,201,408 (2.67%)


/Users/xina/dev/Spam_Classifier/.venv/lib/python3.10/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/Users/xina/dev/Spam_Classifier/.venv/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Now let’s set the training arguments. We'll use `SFTConfig` from the TRL library, which is a wrapper around the standard `TrainingArguments`. We keep epochs, batch size, and sequence length modest to finish training quickly.

In [36]:
ds["train"]

Dataset({
    features: ['prompt', 'completion', 'word', 'Spelling'],
    num_rows: 55
})

In [37]:
# Train the model for a few epochs using SFT before GRPO as in certain cases
# they can work together synergystically.
# See: https://arxiv.org/html/2507.08267v1
# No changes needed here

output_dir = "data/model"

training_args = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=10,
    learning_rate=5 * 1e-4,
    logging_steps=20,
    eval_strategy="steps",
    eval_steps=20,
    save_strategy="no",
    report_to=[],
    fp16=False,
    lr_scheduler_type="cosine",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    args=training_args,
)
trainer.train()

proportion_correct = 0.0

for example in ds["train"].select(range(20)):
    prompt = example["prompt"]
    spelling = example["Spelling"]
    result = check_spelling(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        actual_spelling=spelling,
        max_new_tokens=20,
    )
    proportion_correct += result

print(f"{proportion_correct}/20.0 words correct")

Truncating eval dataset: 100%|██████████| 7/7 [00:00<00:00, 2831.80 examples/s]
/Users/xina/dev/Spam_Classifier/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
20,1.080800,0.862686,2.367684,6833.000000,0.693884
40,0.530200,0.747541,2.097240,13603.000000,0.741263
60,0.431000,0.738365,2.042012,20407.000000,0.741263


Proposed: T-I-R-U-M-P-H. | Actual: T-R-I-U-M-P-H | Matches: ❌
Proposed: S-A-P-I-C-R-H. | Actual: S-A-P-P-H-I-R-E | Matches: ❌
Proposed: E-X-P-S-E-T. | Actual: E-X-P-O-S-E | Matches: ❌
Proposed: F-S-R-E-S-C-O-S. | Actual: F-R-E-S-C-O-S | Matches: ❌
Proposed: W-I-P-S. | Actual: W-I-S-P | Matches: ❌
Proposed: M-I-R-E-G. | Actual: M-I-R-A-G-E | Matches: ❌
Proposed: I-V-O-R-Y. | Actual: I-V-O-R-Y | Matches: ❌
Proposed: O-N-S-H-O-R-D. | Actual: O-N-S-E-T | Matches: ❌
Proposed: E-L-E-U-D. | Actual: E-L-U-D-E | Matches: ❌
Proposed: S-P-H-I-N-X. | Actual: S-P-H-I-N-X | Matches: ❌
Proposed: B-R-A-N-Y. | Actual: B-R-A-W-N | Matches: ❌
Proposed: G-S-O-P-I-Y. | Actual: G-O-S-S-I-P-Y | Matches: ❌
Proposed: E-N-C-H-A-N-T. | Actual: E-N-C-H-A-N-T | Matches: ❌
Proposed: T-A-A-N-R. | Actual: T-A-V-E-R-N | Matches: ❌
Proposed: W-H-I-S-E. | Actual: W-H-I-S-T-L-E | Matches: ❌
Proposed: C-U-P-H-E-R-T. | Actual: C-A-P-T-U-R-E | Matches: ❌
Proposed: E-C-H-O-R-E. | Actual: E-C-H-O | Matches: ❌
Proposed: M-I-R-

The number of words has slightly increased. Let's try training using GRPO now.

First let's create some reward functions.

### Check_spelling()
This function:
- Runs the model
- Generates text
- Extracts the spelling
- Compares it to the ground truth
- Prints diagnostics
- Returns an accuracy-like score

### proportion_correct 
Inputs:
- word → ground truth source
- proposed_spelling → model output (already generated)

Output:
- A single scalar score

In [44]:
correct_spelling = "-".join("hello".upper()) 
proposed_spelling = "H-E-L-L-O!"


In [45]:
max_len = max(len(correct_spelling), len(proposed_spelling))
max_len

10

In [46]:
proposed_spelling_padded = proposed_spelling.ljust(max_len, " ")
proposed_spelling_padded

'H-E-L-L-O!'

In [50]:
correct_spelling_padded = correct_spelling.ljust(max_len, " ")
correct_spelling_padded

'H-E-L-L-O '

In [51]:
# Student Task: Create a helper function proportion_correct that takes a word and
# a proposed spelling from the LLM and returns a score where every matched character
# adds +1 and every  mismatched character subtracts 1 from the reward--including the
# hyphens.
# TODO: Replace occurences of **********

import re


def proportion_correct(word, proposed_spelling):
    correct_spelling = "-".join(word).upper()

    score = 0.0

    # Pad to the same length to handle extra characters
    max_len = max(len(correct_spelling), len(proposed_spelling))
    proposed_spelling_padded = proposed_spelling.ljust(max_len, " ")
    correct_spelling_padded = correct_spelling.ljust(max_len, " ")

    for a, b in zip(correct_spelling_padded, proposed_spelling_padded):
        # Add 1 for matched characters, and subtract one for mismatched
        # **********
        if a == b:
            score += 1
        else:
            score -= 1


    return score / (
        len(correct_spelling)
    )  # Normalize by length of spelling, including dashes


assert proportion_correct("hello", "H-E-L-L-O") == 9 / 9
assert proportion_correct("hello", "H-E-L-") == 3 / 9
assert proportion_correct("hello", "H-E-L-L-O!") == 8 / 9


In [54]:
# Create a `reward_spelling` function that receives a batch of completions and the associated word values
# No changes needed here

import numpy as np


def reward_spelling(completions, word, **kwargs):
    """Reward function that rewards completions with more unique letters."""

    completion_strings = [
        completion.split("\n")[0].strip() for completion in completions
    ]
    print("Completion strings:", completion_strings)
    words = [w for w in word]

    rewards = [proportion_correct(w, c) for w, c in zip(words, completion_strings)]

    # When training, GRPO will pass multiple completions and words to this function.
    # We print just the first one to observe what is happening under the hood.
    print("=====")
    print(
        "Completion example first line:",
        words[0],
        "->",
        completion_strings[0].strip().split("\n")[0].strip(),
    )
    print(f"Spelling mean and std: {np.mean(rewards):.3f} +/- {np.std(rewards):.3f}")
    return rewards


assert reward_spelling(
    completions=[
        "H-E-L-L-O",
        "H-E-L-",
        "H-E-L-L-O!",
    ],
    word=[
        "hello",
        "hello",
        "hello",
    ],
) == [1, 3 / 9, 8 / 9]

Completion strings: ['H-E-L-L-O', 'H-E-L-', 'H-E-L-L-O!']
=====
Completion example first line: hello -> H-E-L-L-O
Spelling mean and std: 0.741 +/- 0.292


In [55]:
# Student task: Create a reward of 1.0 for completions starting with a string
# formatted like X-Y-Z else return 0.0
# TODO: Replace sections marked with **********


def reward_response_in_form_of_letter_dash_letter(completions, word, **kwargs):
    """Reward function that gives a bonus for completions in the form of LETTER-DASH-LETTER."""
    pattern = re.compile(r"^([A-Z]-)+[A-Z]")  # Pattern for LETTER-DASH-LETTER

    words = [w for w in word]

    # Normalize the completions, taking the first line and removing extra whitespace
    completion_strings = [
        completion.split("\n")[0].strip() for completion in completions
    ]

    # Create a list of rewards corresponding to completions
    # Each completion that matches the pattern should receive 1.0,
    # else 0.0
    # rewards = [
    #     **********
    # ]

    # <<< START COMPLETION SECTION
    rewards = [
        1.0 if pattern.match(c) else 0.0 for w, c in zip(words, completion_strings)
    ]
    # >>> END COMPLETION SECTION

    print(
        f"Letter-dash-letter rewards mean and std: {np.mean(rewards):.3f} +/- {np.std(rewards):.3f}"
    )
    return rewards


assert reward_response_in_form_of_letter_dash_letter(
    completions=[
        "H-E-L-L-O",
        "hello",
        "Hi!",
    ],
    word=[
        "hello",
        "hello",
        "hello",
    ],
) == [1, 0, 0]

Letter-dash-letter rewards mean and std: 0.333 +/- 0.471


In [56]:
# Student task: Set the GRPOConfig and initialize the trainer
# See: https://huggingface.co/docs/trl/main/en/grpo_trainer
# TODO: Complete the sections with **********

from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    output_dir="data/spelling-grpo",
    max_completion_length=30,
    logging_steps=5,
    learning_rate=5e-4,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    num_generations=4,
    lr_scheduler_type="cosine",
    beta=0.0,
)
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[reward_spelling, reward_response_in_form_of_letter_dash_letter],
    args=training_args,
    train_dataset=ds["train"],
)
trainer.train()

/Users/xina/dev/Spam_Classifier/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Completion strings: ['M-I-R-H.', 'M-I-T-I-N-R-H.', 'M-I-H-R-E.', 'M-I-T.', 'V-E-L-T-V-E.', 'VEL-e-VET.', 'V-L-E-V-T.', 'V-E-L-V-T.']
=====
Completion example first line: mirth -> M-I-R-H.
Spelling mean and std: 0.107 +/- 0.263
Letter-dash-letter rewards mean and std: 0.875 +/- 0.331


Step,Training Loss
5,0.000800
10,0.008000
15,0.042000
20,0.030800
25,-0.019000
30,0.017800
35,-0.011400
40,0.026800
45,0.010000
50,0.045800


Completion strings: ['M-T-N-R-I-L.', 'M-A-T-L-E.', 'M-A-T-R-I-L.', 'M-A-T-N-R-E-L.', 'V-O-T.', 'V-O-T-E.', 'V-O-T-E.', 'V-O-T-E.']
=====
Completion example first line: mantle -> M-T-N-R-I-L.
Spelling mean and std: 0.205 +/- 0.069
Letter-dash-letter rewards mean and std: 1.000 +/- 0.000
Completion strings: ['S-M-U-T-I.', 'S-M-T-U-R-E-V-E-N-A.', 'S-H-U-M-T, A-W-I-R-S-E-L.', 'S-M-U-T-A-N.', 'B-N-A-B-N-O.', 'B-N-A-B-N.', 'B-A-N-B-O-R.', 'B-A-N-N-E-R.']
=====
Completion example first line: summit -> S-M-U-T-I.
Spelling mean and std: -0.057 +/- 0.616
Letter-dash-letter rewards mean and std: 1.000 +/- 0.000
Completion strings: ['A-B-E-U-R-A.', 'A-B-U-R-E.', 'A-B-E-R-V.', 'A-B-E-U-R-E.', 'M-A-Z-E-R.', 'M-E-A-Z.', 'M-A-Z-E.', 'M-A-Z-A-I.']
=====
Completion example first line: absolve -> A-B-E-U-R-A.
Spelling mean and std: 0.214 +/- 0.316
Letter-dash-letter rewards mean and std: 1.000 +/- 0.000
Completion strings: ['I-V-O-R-Y.', 'Ivy-O-R-Y.', 'I-V-O-R-Y.', 'I-V-O-R-U.', 'L-A-N-T-H.', 'L-N-A-N-T-

TrainOutput(global_step=270, training_loss=0.004313909866053749, metrics={'train_runtime': 392.6852, 'train_samples_per_second': 1.401, 'train_steps_per_second': 0.688, 'total_flos': 0.0, 'train_loss': 0.004313909866053749})

Now we define the `SFTTrainer` and run the fine-tuning process.

## Step 5. Evaluate the fine-tuned model

In [57]:
# Evaluate the fine-tuned model on the same training examples
# No changes needed in this cell

proportion_correct = 0.0

for example in ds["train"].select(range(20)):
    prompt = example["prompt"]
    completion = example["completion"]
    result = check_spelling(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        actual_spelling=completion,
        max_new_tokens=20,
    )
    proportion_correct += result

print(f"{proportion_correct}/20.0 words correct")

/Users/xina/dev/Spam_Classifier/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Proposed: T-I-R-U-M-P-H. | Actual: T-R-I-U-M-P-H. | Matches: ❌
Proposed: S-A-P-I-A-R-H. | Actual: S-A-P-P-H-I-R-E. | Matches: ❌
Proposed: E-X-P-O-S-T. | Actual: E-X-P-O-S-E. | Matches: ❌
Proposed: F-R-S-O-C-S. | Actual: F-R-E-S-C-O-S. | Matches: ❌
Proposed: W-I-P-S. | Actual: W-I-S-P. | Matches: ❌
Proposed: M-I-R-A-G-E. | Actual: M-I-R-A-G-E. | Matches: ✅
Proposed: I-V-O-R-Y. | Actual: I-V-O-R-Y. | Matches: ✅
Proposed: O-N-S-I-D. | Actual: O-N-S-E-T. | Matches: ❌
Proposed: E-L-U-D-E. | Actual: E-L-U-D-E. | Matches: ✅
Proposed: S-P-H-I-N-X. | Actual: S-P-H-I-N-X. | Matches: ✅
Proposed: B-R-A-N-Y. | Actual: B-R-A-W-N. | Matches: ❌
Proposed: G-O-S-P-O-I-Y. | Actual: G-O-S-S-I-P-Y. | Matches: ❌
Proposed: E-N-C-A-P-T-N. | Actual: E-N-C-H-A-N-T. | Matches: ❌
Proposed: T-A-A-N-R-U. | Actual: T-A-V-E-R-N. | Matches: ❌
Proposed: W-H-I-C-T-E. | Actual: W-H-I-S-T-L-E. | Matches: ❌
Proposed: C-U-P-A-R-E. | Actual: C-A-P-T-U-R-E. | Matches: ❌
Proposed: E-C-H-O-R. | Actual: E-C-H-O. | Matches: ❌
Pro

The model now performs better on the training data it has seen. But has it generalized? Let's check its performance on the unseen test set.

In [58]:
# Evaluate the fine-tuned model on the unseen test set
# No changes needed in this cell

proportion_correct = 0.0
num_examples = len(ds["test"])

for example in ds["test"]:
    prompt = example["prompt"]
    completion = example["completion"]
    result = check_spelling(
        model=model,
        tokenizer=tokenizer,
        prompt=prompt,
        actual_spelling=completion,
        max_new_tokens=20,
    )
    proportion_correct += result

print(f"{proportion_correct}/{num_examples}.0 words correct")

Proposed: W-R-I-Y-L-Y. | Actual: W-R-Y-L-Y. | Matches: ❌
Proposed: G-I-L-N-E-S. | Actual: G-L-I-S-T-E-N. | Matches: ❌
Proposed: C-A-S-E-R. | Actual: Q-U-E-S-T. | Matches: ❌
Proposed: C-E-R-E-V-E. | Actual: C-R-A-V-E. | Matches: ❌
Proposed: L-U-S-I-O. | Actual: L-U-S-H. | Matches: ❌
Proposed: F-A-L-I-C-T. | Actual: F-A-B-L-E. | Matches: ❌
Proposed: K-N-A-R-O-N. | Actual: K-N-A-C-K. | Matches: ❌
2.2249999999999996/7.0 words correct


It looks like it has improved! Perhaps with a larger dataset and more training, it could get even better.

## Congratulations for completing the exercise! 🎉

✅ You did it! You successfully fine-tuned a small language model using PEFT with LoRA to teach it a new skill: spelling! You saw how the base model failed completely at the task, and with a very small amount of data and a short training run, the model started to learn how to spell.

<br /><br /><br /><br /><br /><br /><br /><br /><br />